In [3]:
import os
import sys
import math
import numpy as np
import pandas as pd
from itertools import islice

import networkx as nx

from pyspark import SparkContext
from pyspark.streaming import StreamingContext

from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.clustering import StreamingKMeans

%matplotlib widget

import matplotlib.pyplot as plt

In [4]:
#os.environ['PYSPARK_PYTHON'] = '/koko/system/anaconda/envs/python38/bin/python3'
print(os.environ['PYSPARK_PYTHON'] )

/koko/system/anaconda/envs/python38/bin/python3


In [5]:
#os.environ['PYSPARK_DRIVER_PYTHON'] = '/koko/system/anaconda/envs/python38/bin/python3'
print(os.environ['PYSPARK_DRIVER_PYTHON'] )

/koko/system/anaconda/envs/python38/bin/python3


In [26]:
SparkContext.setSystemProperty('spark.executor.memory', '52g')
SparkContext.setSystemProperty('spark.app.name', 'stars')
ssc = StreamingContext(sc,1) 
spark

In [27]:
Gt = nx.Graph()
directed_Gt = nx.DiGraph()

In [28]:
def map_for_vector(line):
    vector = [float(x) for x in line.split(',')]
    return vector

def node_filter(n) :
    
    if n[0][0] >= n[1][0] : # filter out redundant pairs
        return False
    
    ra_1 = n[0][2]
    ra_2 = n[1][2]
    
    d_1 = n[0][3]
    d_2 = n[1][3]

    # 𝛾≈ sqrt([(𝛼𝑎−𝛼𝑏)cos((𝛿𝑎 + 𝛿𝑏) / 2)]2+(𝛿𝑎−𝛿𝑏)2)
    ra_diff = (ra_1 - ra_2)
    d_diff = (d_1 - d_2)
    d_avg = (d_1 + d_2) / 2
    
    distance = math.sqrt( ((ra_diff * math.cos(d_avg)) ** 2) + (d_diff ** 2) )
    if distance < 400: # filter with distance
        
        return True
    
    else :
        return False
    
    # return True
    
    
def add_edges(n):
    Gt.add_edge(n[0][0], n[1][0]) # adding edge from A to B
    print("--- Adding edge: (", n[0][0], n[1][0], ") ---")
    return (n[0][0], 1) # return A, 1 : meaning, the A has a neighbor


def takeAndPrint(rdd, filename):
    taken = rdd.collect()
    for record in taken:
        print("--" + str(record) + "--\n")
        with open(filename, "a") as myfile:
            myfile.write(str(record) + "\n")

In [29]:
batchsize = 10000

#batch_file_folder =  "/common/home/sdb202/project/temp/"
batch_file_folder =  "/common/home/milky-way/temp2/"

In [30]:
star_tile_batches = ssc.textFileStream(batch_file_folder)\
                        .mapPartitionsWithIndex(lambda idx, it: islice(it, 1, None) if idx == 0 else it)\
                        .map(map_for_vector)
                        #.window(1000,1000)

In [31]:
star_pairs = star_tile_batches.transform(lambda rdd: rdd.cartesian(rdd))
filtered_star_pairs = star_pairs.filter(node_filter)
filtered_star_pairs.pprint()

In [32]:
degree_rdd = filtered_star_pairs.map(add_edges) \
                                .reduceByKey(lambda val1, val2: val1 + val2) #count degree of each node        


In [33]:
# star_tile_batches.pprint(5)

In [34]:
ssc.start()

In [35]:
ssc.awaitTerminationOrTimeout(60)

-------------------------------------------
Time: 2020-12-11 06:37:23
-------------------------------------------

-------------------------------------------
Time: 2020-12-11 06:37:24
-------------------------------------------

-------------------------------------------
Time: 2020-12-11 06:37:25
-------------------------------------------

-------------------------------------------
Time: 2020-12-11 06:37:26
-------------------------------------------

-------------------------------------------
Time: 2020-12-11 06:37:27
-------------------------------------------

-------------------------------------------
Time: 2020-12-11 06:37:28
-------------------------------------------

-------------------------------------------
Time: 2020-12-11 06:37:29
-------------------------------------------

-------------------------------------------
Time: 2020-12-11 06:37:30
-------------------------------------------

-------------------------------------------
Time: 2020-12-11 06:37:31
----------

False

In [36]:
ssc.stop(stopSparkContext=False, stopGraceFully=True)

-------------------------------------------
Time: 2020-12-11 06:38:23
-------------------------------------------

-------------------------------------------
Time: 2020-12-11 06:38:24
-------------------------------------------



In [ ]:
# nx.write_gpickle(Gt, "test.gpickle")
# Gt = nx.read_gpickle("test.gpickle")

In [25]:
filtered_star_pairs.for
Gt.nodes()

NodeView(())

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
dschema = StructType()\
                .add("row_id", "double")\
                .add("source_id", "double")\
                .add("ra", "double")\
                .add("d", "double")\
                .add("l", "double")\
                .add("b", "double")\
                .add("j", "double")\
                .add("k", "double")\

dfCSV = spark.readStream.option("sep", ",").option("header", "false").schema(dschema).csv(batch_file_folder)
dfCSV.createOrReplaceTempView("stars")
totalSalary = spark.sql("select ra,sum(d) from salary group by ra")
query = totalSalary.writeStream.outputMode("complete")\
    .option("checkpointLocation", "hello")\
    .format("memory")\
    .queryName('kothi')\
    .start()

query.awaitTermination(60)

False